In [487]:
import nltk
import numpy as np
from collections import Counter
import pandas as pd
import os
from nltk.corpus import wordnet as wn
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# WordNet: Sense, Definition, Example

In [488]:
#https://pythonprogramming.net/wordnet-nltk-tutorial/
word="constant"
for i in wn.synsets(word):
  print(i,"\nDEF: ",i.definition(),"\nEXAMPLES",i.examples())#,wn.synset('book.n.01').examples()

Synset('constant.n.01') 
DEF:  a quantity that does not vary 
EXAMPLES []
Synset('constant.n.02') 
DEF:  a number representing a quantity assumed to have a fixed value in a specified mathematical context 
EXAMPLES ['the velocity of light is a constant']
Synset('changeless.s.02') 
DEF:  unvarying in nature 
EXAMPLES ['maintained a constant temperature', 'principles of unvarying validity']
Synset('constant.a.02') 
DEF:  steadfast in purpose or devotion or affection 
EXAMPLES ['a man constant in adherence to his ideals', 'a constant lover', 'constant as the northern star']
Synset('ceaseless.s.01') 
DEF:  uninterrupted in time and indefinitely long continuing 
EXAMPLES ['the ceaseless thunder of surf', 'in constant pain', 'night and day we live with the incessant noise of the city', 'the never-ending search for happiness', 'the perpetual struggle to maintain standards in a democracy', "man's unceasing warfare with drought and isolation", 'unremitting demands of hunger']


#SemCor

In [489]:
#https://github.com/google-research-datasets/word_sense_disambigation_corpora
#https://www.kaggle.com/saeed1507100/word-sense-disambiguation-1
#http://www.nltk.org/howto/wsd.html
#https://aclweb.org/aclwiki/Word_sense_disambiguation_resources
!git clone https://github.com/nvshrao/Temp.git

fatal: destination path 'Temp' already exists and is not an empty directory.


In [0]:
def parselex(lex):
  lemma,lexsn = lex.split("%")
  i=lexsn.split(":")
  pos=["_","n","v","a","r","s"]
  pos=pos[int(i[0])]
  lex_id = i[2]
  return lemma+"."+pos+"."+lex_id

def parseentry(sentence,prints=True):
  """
  returns Sentence, senseid of each word, corresonding sense, all senses of the word, all corresponding definitions
  may return error
  """
  sent=[]
  sens=[]
  for entry in sentence.split("\n"):
    if "</wf>" in entry:
      word = entry.split('">')[1].split("</wf>")[0]
      sent.append(word)
      if "lemma=" in entry:
        lemma=entry.split('lemma="')[1].split('"')[0]
        lexsn=entry.split('lexsn="')[1].split('"')[0]
        sense=lemma+"%"+lexsn
        sens.append(sense)
      else:
        sens.append("_")
    elif "<punc>" in entry:
      sent.append(entry.strip("<punc>").strip("</punc>"))
      sens.append("_")
    else:
      if prints==True:
        print(entry)
  syn = [wn.synset(parselex(i))  if "_" not in i else "_" for i in sens]
  syndef = [wn.synset(parselex(i)).definition()  if "_" not in i else "_" for i in sens]
  assert len(sent)==len(syn)
  other=[]
  otherdef=[]
  #sentence, senses= parseentry(sents[3])
  for i,word in enumerate(sent):
    if str(syn[i])!="_":
      other_senses=wn.synsets(word)
      other.append(other_senses)
      otherdef.append([i.definition() for i in other_senses])
    else:
      other.append("_")
      otherdef.append("_")
  return sent,syn,syndef,other,otherdef

def readfile(filename,prints=False):
  file=open(filename).read()
  paras=file.split("<p pnum=")
  if prints==True:
    print(len(paras))
  sents=[i.split("<s snum=") for i in paras]
  sents = [j for i in sents for j in i]# Ignore first two sents mostly, check last one
  if prints==True:
    print(len(sents))
  sents = [i for i in sents if "</wf>" in i]# only those with at least a word
  if prints==True:
    print(len(sents))
  return sents

In [491]:
path = "./Temp/data/semcor/brown1/tagfiles/"
brown1files = os.listdir(path)
f = [readfile(path+i) for i in brown1files]
brown1 = [j for i in f for j in i]

path = "./Temp/data/semcor/brown2/tagfiles/"
brown2files = os.listdir(path)
f = [readfile(path+i) for i in brown2files]
brown2 = [j for i in f for j in i]

path = "./Temp/data/semcor/brownv/tagfiles/"
brownvfiles = os.listdir(path)
f = [readfile(path+i) for i in brownvfiles]
brownv = [j for i in f for j in i]

allsentences=brown1+brown2+brownv
print(len(brown1),len(brown2),len(brownv),len(allsentences))

11178 8954 17036 37168


In [492]:
errors=[]
sentences=[]
synset=[]
sense=[]
allsynsets=[]
allsenses=[]
for i,j in enumerate(allsentences):
  try:
    a,b,c,d,e = parseentry(j,prints=False)
    sentences.append(a)
    synset.append(b)
    sense.append(c)
    allsynsets.append(d)
    allsenses.append(e)
  except:
    errors.append(i)
assert len(sentences) == len(synset) == len(sense) == len(allsynsets) == len(allsenses)
len(errors),len(allsentences)

(3820, 37168)

In [0]:
#wn.synsets("historical")
#wn.synset('historical.s.00')

In [494]:
data = {'sentences':sentences, 'synset':synset,"sense":sense,"allsynsets":allsynsets, "allsenses":allsenses}
df = pd.DataFrame(data)
df.head()

,sentences,synset,sense,allsynsets,allsenses
0,"[Andy, did, not, see, the, newspapers, the, ne...","[Synset('person.n.03'), _, Synset('not.r.01'),...",[a grammatical category used in the classifica...,"[[], _, [Synset('not.r.01')], [Synset('see.n.0...","[[], _, [negation of a word or group of words]..."
1,"[Someone, on, his, staff, -, he, suspected, it...","[Synset('person.n.01'), _, _, Synset('staff.n....","[a human being, _, _, personnel who assist the...","[[Synset('person.n.01')], _, _, [Synset('staff...","[[a human being], _, _, [personnel who assist ..."
2,"[Memory, flooded, him, the, instant, he, opene...","[Synset('memory.n.02'), Synset('flood.v.04'), ...",[the cognitive processes whereby past experien...,"[[Synset('memory.n.01'), Synset('memory.n.02')...","[[something that is remembered, the cognitive ..."
3,"[Outside, his, window, bloomed, a, beautiful, ...","[_, _, Synset('window.n.08'), Synset('bloom.v....","[_, _, (computer science) a rectangular part o...","[_, _, [Synset('window.n.01'), Synset('window....","[_, _, [a framework of wood or metal that cont..."
4,"[Presumably, the, same, sun, was, shining, upo...","[Synset('presumably.r.01'), _, Synset('same.a....","[by reasonable assumption, _, closely similar ...","[[Synset('presumably.r.01')], _, [Synset('lapp...","[[by reasonable assumption], _, [a member of a..."


# Pytorch BERT

In [495]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import expit
import torch
!pip install tensorflow-gpu
!pip install transformers
import transformers

In [0]:
model_class = transformers.BertModel
tokenizer_class = transformers.BertTokenizer
model_name = 'bert-base-uncased'
tokenizer = tokenizer_class.from_pretrained(model_name)
bertmodel = model_class.from_pretrained(model_name,output_hidden_states=True,output_attentions=True)

#padded_seq=tokenizer.encode("Hi My name is anvesh")
#input_ids = torch.tensor(np.array(padded_seq))
#with torch.no_grad():
#    torch_output = model(input_ids)
#last_hidden_state,pooler_output,hidden_states,attentions = torch_output
#last_hidden_state.shape, pooler_output.shape, torch.stack(hidden_states).shape, torch.stack(attentions).shape
#assert (hidden_states[-1] == last_hidden_state).all()

T=30
def tknz(sentence,T=30):
  return tokenizer.encode(sentence,add_special_tokens = True,max_length = T,pad_to_max_length=True)

def get_representation(sentence,model=bertmodel):
  if type(sentence) is list:
    sentence=" ".join(sentence)
  assert type(sentence)==str
  padded_seq = tknz(sentence)
  num_pads = padded_seq.count(0) if padded_seq.count(0) != 0 else -T-1#  -0 wont work if no pads
  padded_seq=[padded_seq]
  input_ids = torch.tensor(np.array(padded_seq))
  with torch.no_grad():
    torch_output = model(input_ids)
  last_hidden_state, pooler_output, hidden_states, attentions = torch_output
  #return np.array(last_hidden_state)[0][:-num_pads][1:-1] # SEP and CLS are to be ignored
  return np.array(last_hidden_state)[0][:-num_pads] #because only 1 sentence and avoid padding representations

In [0]:
def findword(word,sentence):
  """ Will only work if word hasn't been split in more than two tokens """
  word=word.lower()
  flag=0
  s = " ".join(sentence).replace("_"," ")
  en = tokenizer.encode(s,max_length = 30,pad_to_max_length=True)
  bert_tokenized_sentence=tokenizer.convert_ids_to_tokens(en)
  if word in bert_tokenized_sentence:
    return [bert_tokenized_sentence.index(word)]
  else:
    for i,w in enumerate(bert_tokenized_sentence):
      if "##" in w:
        if bert_tokenized_sentence[i-1]+bert_tokenized_sentence[i].replace("##","") == word:
          return [i-1,i]
  return -1# could not find

def WSD(word,sentence):
  """
  sentence needs to be a list of tokens and word needs to be present in the list
  return correct wn sense and defination
  """
  word_indices = findword(word,sentence)
  if word_indices == -1:
    print("Couldn't find word in sentence.")
    return
  syns= wn.synsets(word)
  if len(syns)<=1:
    print("Word does not have enough senses.")
    return 
  all_senses = [i.definition() for i in syns]
  whole = get_representation(sentence)
  word_vector = np.mean(np.take(whole,word_indices,axis=0),axis=0)
  all_senses_vectors = [np.mean(get_representation(i),axis=0) for i in all_senses]
  distances = [np.linalg.norm(word_vector-i) for i in all_senses_vectors]
  minindex = np.argmin(distances)
  assert(len(distances)==len(syns))
  sorted_syns = [x for _,x in sorted(zip(distances,syns))]
  return sorted_syns,sorted_syns[0].definition() 

In [498]:
word ='bank'
sentence = ['He', 'sat', 'at', 'the',"river",'bank',"."]
WSD(word,sentence)

([Synset('bank.n.01'),
  Synset('depository_financial_institution.n.01'),
  Synset('deposit.v.02'),
  Synset('bank.n.06'),
  Synset('bank.n.09'),
  Synset('bank.v.02'),
  Synset('savings_bank.n.02'),
  Synset('bank.v.03'),
  Synset('bank.v.05'),
  Synset('bank.n.10'),
  Synset('bank.n.03'),
  Synset('bank.v.04'),
  Synset('bank.n.05'),
  Synset('bank.n.04'),
  Synset('bank.v.07'),
  Synset('bank.n.07'),
  Synset('trust.v.01'),
  Synset('bank.v.01')],
 'sloping land (especially the slope beside a body of water)')

In [499]:
word ='bank'
sentence = ['He', 'worked', "as","a","cashier",'at', 'the',"bank","."]
WSD(word,sentence)

([Synset('depository_financial_institution.n.01'),
  Synset('bank.v.03'),
  Synset('deposit.v.02'),
  Synset('bank.n.06'),
  Synset('savings_bank.n.02'),
  Synset('bank.v.05'),
  Synset('bank.n.09'),
  Synset('bank.v.02'),
  Synset('bank.n.01'),
  Synset('bank.v.04'),
  Synset('bank.n.05'),
  Synset('bank.n.04'),
  Synset('bank.n.07'),
  Synset('bank.n.10'),
  Synset('bank.v.07'),
  Synset('trust.v.01'),
  Synset('bank.n.03'),
  Synset('bank.v.01')],
 'a financial institution that accepts deposits and channels the money into lending activities')

In [500]:
word ='currents'
sentence = ['The', 'currents', "in","the","river","were","strong","."]
WSD(word,sentence)

([Synset('stream.n.02'), Synset('current.n.01'), Synset('current.n.02')],
 'dominant course (suggestive of running water) of successive events or ideas')

In [501]:
word ='current'
sentence = ['current', 'events', "show","the","pandemic","clearly","."]
WSD(word,sentence)

([Synset('current.a.01'),
  Synset('current.n.01'),
  Synset('stream.n.02'),
  Synset('current.n.02')],
 'occurring in or belonging to the present time')